In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('EUR_USD_15Min.csv',index_col=0) # assuming the data is in a CSV file
df.head()
df=df.drop('DateTime',axis=1)
data=df= df.dropna()
df.head()

,open,close,low,high
time,,,,
2015-01-01 22:00:00+00:00,1.21057,1.21050,1.21027,1.21074
2015-01-01 22:15:00+00:00,1.21048,1.21030,1.20995,1.21075
2015-01-01 22:30:00+00:00,1.21033,1.21042,1.21018,1.21042
2015-01-01 22:45:00+00:00,1.21039,1.21022,1.21007,1.21039
2015-01-01 23:00:00+00:00,1.21020,1.21007,1.20990,1.21020


In [ ]:
def label_data(df,lookback=8, ma_period=20, range_pct=0.02, rsi_period=14, rsi_overbought=70, rsi_oversold=30, bb_period=20, bb_std=2):
    # Calculate the Moving Average
    ma = df['close'].rolling(window=ma_period).mean()

    # Calculate the price range
    high_range = ma * (1 + range_pct)
    low_range = ma * (1 - range_pct)

    # Calculate the RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Calculate the Bollinger Bands
    std = df['close'].rolling(window=bb_period).std()
    upper_band = ma + bb_std * std
    lower_band = ma - bb_std * std
    df["rsi"] = rsi
    df["up"] = upper_band
    df["down"] = lower_band
    # Create the labels
    labels = [0]*lookback
    for i in range(lookback,len(df)):
        if df['close'][i] >  df['close'][i-lookback:i].max() and rsi[i] > rsi_overbought and df['close'][i] > upper_band[i]:
            labels.append(1)
        elif df['close'][i] < df['close'][i-lookback:i].min() and rsi[i] < rsi_oversold and df['close'][i] < lower_band[i]:
            labels.append(-1)
        else:
            labels.append(0)

    # Add the labels to the DataFrame
    df['Label'] = labels

    return df
data= label_data(data)    
print(len(data))

83297


In [ ]:
data["pattern"] = ""
from tqdm import tqdm
for i in (range(5,len(data))) :
  a = str((data["close"].iloc[i]>data["close"].iloc[i-1])*1)
  a = a+ str((data["close"].iloc[i-1]>data["close"].iloc[i-2])*1)
  a = a+ str((data["close"].iloc[i-2]>data["close"].iloc[i-3])*1)
  a = a+ str((data["close"].iloc[i-3]>data["close"].iloc[i-4])*1)
  a = a+ str((data["close"].iloc[i-4]>data["close"].iloc[i-5])*1)
  data.pattern.iloc[i] = a

data = data.iloc[5:]


100%|██████████| 83292/83292 [00:51<00:00, 1625.88it/s]


In [ ]:
data["Label"].value_counts()


 0    78965
 1     2168
-1     2159
Name: Label, dtype: int64

In [ ]:
#data["Label"].value_counts()
data.tail()

,open,close,low,high,rsi,up,down,Label,pattern
time,,,,,,,,,
2018-05-09 20:45:00+00:00,1.18532,1.18512,1.18505,1.18532,37.921348,1.186853,1.184405,0,01000
2018-05-09 21:00:00+00:00,1.18495,1.18510,1.18468,1.18533,35.549133,1.186845,1.184428,0,00100
2018-05-09 21:15:00+00:00,1.18510,1.18518,1.18506,1.18528,35.549133,1.186846,1.184398,0,10010
2018-05-09 21:30:00+00:00,1.18510,1.18523,1.18500,1.18523,26.885246,1.186816,1.184475,0,11001
2018-05-09 21:45:00+00:00,1.18524,1.18522,1.18507,1.18544,29.496403,1.186816,1.184447,0,01100


In [ ]:
! pip install catboost
from catboost import CatBoostClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

tscv = TimeSeriesSplit(n_splits=5,gap=0)
#calling the oversampling function
ros = RandomOverSampler()
X = data.drop("Label",axis=1) #Featues
y = data.Label  #Target
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print("######### FOLD " + str(i) + "########")
    
    X_train,y_train = X.iloc[train_index] , y.iloc[train_index]
    X_test,y_test = X.iloc[test_index] , y.iloc[test_index]
    #oversampling the training data only
    X_trainR,y_trainR = ros.fit_resample(X_train,y_train)
    dt = clf = CatBoostClassifier(
    iterations=100, 
    learning_rate=0.05, 
    #loss_function='CrossEntropy'
    )
    dt.fit(X_trainR, y_trainR ,   eval_set=(X_test, y_test), 
        verbose=False,early_stopping_rounds=15,use_best_model=True
)
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix",cm)
    print('                        ')
    f1 = f1_score(y_test, y_pred,average="weighted")
    print('Test F1 score:', round(f1,2))
    print('                        ')
    f1 = f1_score(y_train, dt.predict(X_train),average="weighted")
    print('Train F1 score:',round(f1,2))
    print("#############################################")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
######### FOLD 0########
              precision    recall  f1-score   support

          -1       0.34      1.00      0.50       344
           0       1.00      0.90      0.95     13167
           1       0.35      1.00      0.52       371

    accuracy                           0.90     13882
   macro avg       0.56      0.97      0.66     13882
weighted avg       0.97      0.90      0.92     13882

Confusion Matrix [[  344     0     0]
 [  675 11798   694]
 [    0     0   371]]
                        
Test F1 score: 0.92
                        
Train F1 score: 0.93
#############################################
######### FOLD 1########
              precision    recall  f1-score   support

          -1       0.35      0.99      0.51       331
           0       1.00      0.91      0.95     13208
           1       0.40      1.00      0.57       343

    accuracy                      